In [65]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [161]:
#Reading in all data, tab delimited
playByPlayDf = pd.read_csv('NBA Hackathon - Play by Play Data Sample (50 Games).txt',sep='\t')
lineupsDf = pd.read_csv('NBA Hackathon - Game Lineup Data Sample (50 Games).txt',sep='\t')
codesDf = pd.read_csv('NBA Hackathon - Event Codes.txt',sep='\t')

In [162]:
def find_code(msg_type,action_type=None):
    if action_type:
        return codesDf[(codesDf['Event_Msg_Type'] == msg_type) & (codesDf['Action_Type']==action_type)]
    else:
        return codesDf[codesDf['Event_Msg_Type'] == msg_type]

In [163]:
codesDf.Event_Msg_Type.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

##### Code descriptions
1: Made Shot <br>
2: Missed Shot <br>
3: Free Throw (Except Action Type == 0)<br>
6: Foul<br>
8: Substitution<br>

###### Sort order for play by play per instructions
Period (ascending), PC_Time (descending), WC_Time (ascending),
Event_Number (ascending)

In [164]:
print(lineupsDf.columns)
print('')
print(playByPlayDf.columns)

Index(['Game_id', 'Period', 'Person_id', 'Team_id', 'status'], dtype='object')

Index(['Game_id', 'Event_Num', 'Event_Msg_Type', 'Period', 'WC_Time',
       'PC_Time', 'Action_Type', 'Option1', 'Option2', 'Option3', 'Team_id',
       'Person1', 'Person2', 'Team_id_type'],
      dtype='object')


In [165]:
playByPlayDf.sort_values('Period',ascending=True,inplace=True)
playByPlayDf.sort_values('PC_Time',ascending=False,inplace=True)
playByPlayDf.sort_values('WC_Time',ascending=True,inplace=True)
playByPlayDf.sort_values('Event_Num',ascending=True,inplace=True)

In [191]:
stacked = pd.DataFrame()
for gameId in lineupsDf['Game_id'].unique():
#     gameId = '021fd159b55773fba8157e2090fe0fe2'
    gameLineups = lineupsDf[lineupsDf['Game_id'] == gameId]
    gamePbp = playByPlayDf[playByPlayDf['Game_id'] == gameId]
    gamePbp = gamePbp[gamePbp['Team_id'].isin(gameLineups['Team_id'].unique())]
    for team in gameLineups['Team_id'].unique():
        currTeam = set(gameLineups[gameLineups['Team_id'] == team]['Person_id'].unique())
        lastIter=set()
        while len(currTeam) != len(lastIter):
            lastIter=deepcopy(currTeam)
            currTeam.update(
                gamePbp[
                    (gamePbp['Event_Msg_Type'] == 8) & 
                    (gamePbp['Person1'].isin(currTeam))
                ]['Person2'].unique()
            )
        teamCount = len(currTeam)
        currDf = pd.DataFrame(
            list(zip(currTeam,[0] * teamCount, [gameId] * teamCount,[team] * teamCount)),
            columns=['playerid','plus_minus','gameid','teamid']
        )
        stacked = pd.concat([stacked,currDf])

In [207]:
find_code(3)

,Event_Msg_Type,Action_Type,Event_Msg_Type_Description,Action_Type_Description
65,3,10,Free Throw,Free Throw 1 of 1
66,3,11,Free Throw,Free Throw 1 of 2
67,3,12,Free Throw,Free Throw 2 of 2
68,3,13,Free Throw,Free Throw 1 of 3
69,3,15,Free Throw,Free Throw 3 of 3
70,3,16,Free Throw,Free Throw Technical
71,3,17,Free Throw,Free Throw Clear Path
142,3,14,Free Throw,Free Throw 2 of 3
146,3,0,No Shot,NaN
147,3,21,Free Throw,Free Throw Technical 1 of 2


In [213]:
gamePbp[
#     (gamePbp.Event_Msg_Type == 6) | 
    ((gamePbp.Event_Msg_Type == 3) & (gamePbp.Action_Type.isin([0,21,25,11,13,14,18,27,28]))) 
#     (gamePbp.Event_Msg_Type == 8)
]

,Game_id,Event_Num,Event_Msg_Type,Period,WC_Time,PC_Time,Action_Type,Option1,Option2,Option3,Team_id,Person1,Person2,Team_id_type
22403,fdeb2950c4d5209d449ebd2d6afac11e,73,3,1,554866,2540,11,1,0,0,01be0ad4af7aeb1f6d2cc2b6b6d6d811,c6e0c30c8c02b0706682ec89eb6fc565,6bcf6c1f8c373d25fca1579bc4464a91,2
22438,fdeb2950c4d5209d449ebd2d6afac11e,118,3,1,558341,680,11,2,0,0,3cd0b15957ceb80f5125bef8bd1bbea7,f79a2b5e6190ac3c534645e806f1b611,6bcf6c1f8c373d25fca1579bc4464a91,3
22448,fdeb2950c4d5209d449ebd2d6afac11e,130,3,1,559059,262,11,2,1,0,01be0ad4af7aeb1f6d2cc2b6b6d6d811,07695ef9feebc6ba02831f0f3c22662a,6bcf6c1f8c373d25fca1579bc4464a91,2
22453,fdeb2950c4d5209d449ebd2d6afac11e,137,3,1,559827,44,11,1,0,0,3cd0b15957ceb80f5125bef8bd1bbea7,45e7a758a91a254ff04756562e740f2d,6bcf6c1f8c373d25fca1579bc4464a91,3
22460,fdeb2950c4d5209d449ebd2d6afac11e,148,3,2,562272,7080,11,1,0,0,01be0ad4af7aeb1f6d2cc2b6b6d6d811,c17f6e6b83ccc768962046f1af1372a5,6bcf6c1f8c373d25fca1579bc4464a91,2
22496,fdeb2950c4d5209d449ebd2d6afac11e,191,3,2,566794,5100,11,1,0,0,01be0ad4af7aeb1f6d2cc2b6b6d6d811,c17f6e6b83ccc768962046f1af1372a5,6bcf6c1f8c373d25fca1579bc4464a91,2
22502,fdeb2950c4d5209d449ebd2d6afac11e,200,3,2,567846,4610,11,2,0,0,3cd0b15957ceb80f5125bef8bd1bbea7,4f4e47023263931e1445dc97f7dae941,6bcf6c1f8c373d25fca1579bc4464a91,3
22542,fdeb2950c4d5209d449ebd2d6afac11e,247,3,2,572658,2110,11,1,0,0,3cd0b15957ceb80f5125bef8bd1bbea7,315f46f7a5e1de7ef9b4c2430ce64e2d,6bcf6c1f8c373d25fca1579bc4464a91,3
22547,fdeb2950c4d5209d449ebd2d6afac11e,255,3,2,573115,1970,11,2,1,0,3cd0b15957ceb80f5125bef8bd1bbea7,315f46f7a5e1de7ef9b4c2430ce64e2d,6bcf6c1f8c373d25fca1579bc4464a91,3
22560,fdeb2950c4d5209d449ebd2d6afac11e,272,3,2,576430,920,11,1,0,0,3cd0b15957ceb80f5125bef8bd1bbea7,315f46f7a5e1de7ef9b4c2430ce64e2d,6bcf6c1f8c373d25fca1579bc4464a91,3
